In [1]:
from class_declarations import Email, Conversation
import pickle

from neo4j import GraphDatabase


In [2]:
conversations = pickle.load(open("conversations.pkl", "rb"))

In [27]:
len(conversations)

90605

In [70]:
subconversations = []
for c in conversations:
    if len(c.emails) > 5:
        subconversations.append(c)

In [14]:
smallconversations = conversations[0:1]

In [71]:
len(subconversations)

2484

In [82]:
for i in list(set(subconversations[0].mentioned_links)):
    in

<class 'class_declarations.Link'>
<class 'class_declarations.Link'>


In [15]:
for s in smallconversations:
    print(s)

In [67]:

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "test1"))

In [59]:
def add_conversation(tx, conversation):
    tx.run("MERGE (c:Conversation {id: $h})",
           h=hash(conversation))

In [60]:
def add_person(tx, name, address, org):
    if org == "" or org == None:
        print("org is blank")
        tx.run("MERGE (p:Person {name: $name}) ON CREATE SET p.address = $address",
           name=name, address=address)
    else:
        tx.run("""MERGE (p:Person {name: $name}) ON CREATE SET p.address = $address
           MERGE (o:Organization {name: $org})
           MERGE (p)-[rel:belongs_to]->(o)""",
           name=name, address=address, org=org)
    

In [61]:
def add_relation(tx, sender, receiver):
    tx.run("""MATCH (s:Person {name: $sname}) 
              MATCH (r:Person {name: $rname})
             MERGE (s)-[rel:talked_to]->(r)""", sname=sender.name, rname=receiver.name)

In [65]:
def connect_conversation(tx, person, conversation):
    tx.run("""MATCH (p:Person {name: $name}) 
              MATCH (c:Conversation {id: $h})
              MERGE (p)-[rel:involved_in]->(c)""", name=person, h=hash(conversation))
    

In [85]:
def add_document(tx, doc, conversation):
    tx.run("MERGE (d:Document {docid: $docid})", docid=doc.url)
    tx.run("""MATCH (d:Document {docid: $docid}) 
              MATCH (c:Conversation {id: $h})
              MERGE (c)-[rel:mentions]->(d)""", docid=doc.url, h=hash(conversation))
    

In [86]:
with driver.session() as session:
    for c in conversations:
        if len(c.emails) > 5:
            session.write_transaction(add_conversation, c)
            for link in list(set(c.mentioned_links)):
                session.write_transaction(add_document, link, c)
            for s,r in zip(c.senders, c.receivers):
                if not (s.name == "" or r.name == ""):
                    session.write_transaction(add_person,s.name, s.address, s.org)
                    session.write_transaction(add_person,r.name, r.address, r.org)
                    session.write_transaction(add_relation,s,r)
            for i in c.interlocutors:
                session.write_transaction(connect_conversation, i.name, c)
            

org is blank
